<a href="https://colab.research.google.com/github/Mjboothaus/personal-tan-lea-kuan/blob/main/notebooks/storage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## S3 Object Storage (on Scaleway.com)

75GB of free object storage (S3) storage is available on Scaleway.

https://www.simplecto.com/using-django-and-boto3-with-scaleway-object-storage/

* `ACCESS_KEY_ID` and `SECRET_ACCESS_KEY` can be obtained from the [credentials control panel](https://console.scaleway.com/project/credentials) under API Keys.
* `STORAGE_BUCKET_NAME` is the name of the bucket you create on [objects administration page](https://console.scaleway.com/object-storage/buckets)
* `DEFAULT_ACL` is set to public-read so that the objects can be pulled from a URL without any access keys or time-limited signatures.
* `S3_REGION_NAME` and `S3_ENDPOINT_URL` should be configured so that `boto3` knows to point to the Scaleway resources.

All of these are references in the Scaleway's docs on Object Storage.

### Resources:
* https://www.scaleway.com/en/docs/object-storage-feature/
* https://www.scaleway.com/en/docs/how-to-migrate-object-storage-buckets-with-rclone/
* https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3.html

In [1]:
import tomli
from pathlib import Path

In [5]:
APP_CONFIG_TOML = 'settings.toml'
REPO_ADDRESS = "gh:personal-tan-lea-kuan"

In [6]:
REPO_NAME = REPO_ADDRESS.replace("gh:", "")

In [7]:
REPO_NAME

'personal-tan-lea-kuan'

In [8]:
try:
    with open(Path('..')/APP_CONFIG_TOML, encoding="utf-8") as f:
        app_config = tomli.load(f)
except Exception as e:
    print(e)

[Errno 2] No such file or directory: '../settings.toml'


In [ ]:
ACCESS_KEY_ID = app_config['S3']['ACCESS_KEY_ID']
SECRET_ACCESS_TOML = app_config['S3']['SECRET_ACCESS_TOML']
STORAGE_BUCKET_NAME = app_config['S3']['STORAGE_BUCKET_NAME']
DEFAULT_ACL = app_config['S3']['DEFAULT_ACL']
REGION_NAME = app_config['S3']['REGION_NAME']
ENDPOINT_URL =  app_config['S3']['ENDPOINT_URL']

KeyError: 'SECRET_ACCESS_TOML'

In [ ]:
SECRET_ACCESS_KEY

'app-secrets.toml'

In [ ]:
import boto3
from pathlib import Path

In [ ]:
#from os import path, makedirs
#from botocore.exceptions import ClientError
#from boto3.exceptions import S3TransferFailedError

In [ ]:
s3 = boto3.client('s3', 
        region_name=REGION_NAME, 
        endpoint_url=ENDPOINT_URL, 
        access_key_id=ACCESS_KEY_ID,
        secret_access_key=SECRET_ACCESS_KEY)

TypeError: client() got an unexpected keyword argument 'access_key_id'

In [ ]:
def download_s3_folder(s3, bucket_name, s3_folder, local_dir=None):
    filecount = 0
    files = []
    if not local_dir.exists():
        Path.makedirs(local_dir)
    bucket_list=s3.list_objects(Bucket=bucket_name)['Contents']
    for s3_key in bucket_list:
        s3_object = s3_key['Key']
        if not s3_object.endswith("/"):
            filepath = Path(local_dir)/s3_object
            s3.download_file(bucket_name, s3_object, filepath.as_posix())
            filecount+=1
            files.append(s3_object)
        else:
            if not (Path(local_dir)/s3_object).exists():
                Path.makedirs(Path(local_dir)/s3_object)
    return filecount, files

In [ ]:
filecount, files = download_s3_folder(s3, STORAGE_BUCKET_NAME, REPO_NAME, Path.home()/'tmp')

In [ ]:
print(str(filecount) + ' files downloaded')
print(files)

9 files downloaded
['emmaus-walking/GWW/2020-05-11-095108-Walking-Michael and Ai Leen’s Apple\xa0Watch.fit', 'emmaus-walking/GWW/2020-05-18-110321-Walking-Michael and Ai Leen’s Apple\xa0Watch.fit', 'emmaus-walking/GWW/2020-05-25-095315-Walking-Michael and Ai Leen’s Apple\xa0Watch.fit', 'emmaus-walking/GWW/2020-05-25-115046-Walking-Michael and Ai Leen’s Apple\xa0Watch.fit', 'emmaus-walking/GWW/2020-06-01-094350-Walking-Michael and Ai Leen’s Apple\xa0Watch.fit', 'emmaus-walking/GWW/2020-06-15-084936-Walking-Michael and Ai Leen’s Apple\xa0Watch.fit', 'emmaus-walking/GWW/2020-06-22-085751-Walking-Michael and Ai Leen’s Apple\xa0Watch.fit', 'emmaus-walking/GWW/2021-03-12-085528-Walking-Michael and Ai Leen’s Apple\xa0Watch.fit', 'emmaus-walking/GWW/2021-04-03-084312-Walking-Michael and Ai Leen’s Apple\xa0Watch.fit']
